In [ ]:
# Auto sync for editing python script file in real time
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "EECS595/Emoji"

In [ ]:
%cd /content/drive/My Drive/EECS595/Emoji

In [ ]:
import os
import sys

assert sys.version_info[0] == 3
assert sys.version_info[1] >= 5

GOOGLE_DRIVE_PATH = os.path.join("/content", "drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)

print(os.listdir(GOOGLE_DRIVE_PATH))

assert "Embedding.ipynb" in os.listdir(GOOGLE_DRIVE_PATH), "<Warning>: Embedding.ipynb not found."
# assert "Embedding.py" in os.listdir(
#     GOOGLE_DRIVE_PATH
# ), "<Warning>: Embedding not found."

In [ ]:
!ls

In [ ]:
!pip install transformers
!pip install emoji

In [ ]:
import pandas as pd
from transformers import BertTokenizerFast, BertModel, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from torch import nn
import emoji as emo
from tqdm import tqdm
import concurrent.futures
import multiprocessing as mp
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
import unicodedata as ud
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from itertools import repeat
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

#Naïve sentiment analysis of a tweet

basic algorithm: 
1. if all blatantly positive emoji - positive
2. if all blatantly negative emoji - negative 
3. random assign

#Naïve Classification Pipeline
1. Load rehydrated twitter data 
2. Split into train and test sets
3. v1. Make emoji dictionary from TM Senti; v2. Incorporate EmoTag 620 into the dict
4. Perform sentiment analysis with the alg

#1. Load Data

In [ ]:
emo_pos = ['😀','😃','😄','😁','😆','😊','😇','😌','😍','🥰','🤩','😘','😗','😚','😙','😋','😝','🤗','🤤','🥳','😎','😺','😸','😻','😽','✌️','👍','❤️','💕','♥️','💙','💜','💗','🖤','💛','💖','💞','💚','❣️','💓','🧡','💘','💝','💟','🤍','🤎']
emo_beg = ['🙃','😐','😑','😶','😒','🙄','😬','😔','🤢','🤮','😵‍💫','😕','🙁','☹️','😟','😧','😨','😰','😢','😭','👎','😖','😣','😞','😓','😩','😫','😠','😡','🤬','😿','🖕','😥','💔']

emoticon_pos = [':)', ':-)', ':)', ':D', '=D', ':-]', ':]', ':-3', ':3', ':->', ':>', '8-)', ':-}', ':}', ':o)', ':c)', ':', 'ˆ)', '=]', '=)', ':D', '8D', 'xD', '=D', '=3', 'BDˆ', ':-))', ':’)', ';)', ';)', '*-)', '*)', ';]', ';ˆ)', ':,', ';D', '<3']
emoticon_neg = [':(',':-(',':(', ':’(', ':(', ':c',':<',':<',':[', ':[', ':-||', '>:[', ':{ ',':@', '>:(', 'D’:', 'D:<', 'D:', 'D;', 'D=', ':/', ':/', ':.', '>:/' , '>:/', ':\', '=/', '=\', ':L', '=L', ':S', '</3', '<\']

In [2]:
parquet_dir = './rehydrated_tweets_parquet'

In [ ]:
df = pd.read_parquet('./rehydrated_tweets_parquet/2013/01/en_2013_01_29.parquet')
# pd.read_parquet('en_2013_01_29.parquet', engine='pyarrow')
df

# 3. v1. Make emoji list from TM Senti
Pos2013-2020: 😍🥰😌👍😊😘:) ;) :D
Neg2013-2020: 😭 :( 😔🙄😢😩 :/ 😒

In [3]:
emotm_pos = ['😍', '🥰', '😌', '👍', '😊', '😘', ':)', ';)', ':D']
emotm_neg =['😭', '😔', '🙄', '😢', '😩', '😒', ':/', ':(' ]
print(emotm_pos)
print(emotm_neg)

['😍', '🥰', '😌', '👍', '😊', '😘', ':)', ';)', ':D']
['😭', '😔', '🙄', '😢', '😩', '😒', ':/', ':(']


a set of emojis from df `tweet_text` col

In [ ]:
emoset = set()
v = df['tweet_text'].values
for tweet_str in tqdm(v):
  for char in tweet_str:
    if emo.is_emoji(char):
      emoset.add(char)
emolst = list(emoset)
print(emoset)
print(emolst)

emoticon list 

In [ ]:
emov = df_sm['emojis'].values.tolist()

emoticons = set()
for emoji_str in tqdm(emov):
    if ',' in emoji_str:
        emoji_list = emoji_str.split(',')
        for emoji in emoji_list:
          if not emo.is_emoji(emoji):
            # if not emo.is_emoji(emoji) and emoji not in emoticons:
            emoticons.add(emoji)
    else:
        if not emo.is_emoji(emoji_str):
            emoticons.add(emoji_str)
emoticonslst = list(emoticons)
print(emoticonslst)

somehow this function goes in infiite loops

In [ ]:
emov = df_sm['emojis'].values.tolist()

def extract_emoticons(emov):
  emoticons = set()
  for emoji_str in tqdm(emov):
    if ',' in emoji_str:
        emoji_list = emoji_str.split(',')
        for emoji in emoji_list:
          if not emo.is_emoji(emoji):
            # if not emo.is_emoji(emoji) and emoji not in emoticons:
            emoticons.add(emoji)
    else:
        if not emo.is_emoji(emoji_str):
            emoticons.add(emoji_str)
  emoticonslst = list(emoticons)
  return emoticonslst
  
df['extracted_emoticons'] = df.apply(lambda row : extract_emoticons(row['emojis']), axis = 1)
df['extracted_emoticons'].head(5)

emojis tweet by tweet

In [ ]:
lst = df['tweet_text'].values.tolist()
lstemo = df['emojis'].values.tolist()

def extract_emojis(lst):
  # print(lst)
  extracted_emojis = []
  # extracted_emojis2 = []
  for tweet in lst:
    for word in tweet:
      if emo.is_emoji(word) and word not in extracted_emojis:
        extracted_emojis.append(word)
        # extracted_emojis2 = extracted_emojis + emoticons
  return extracted_emojis

df['extracted_emojis'] = df.apply(lambda row : extract_emojis(row['tweet_text']), axis = 1)
df_sm['extracted_emojis'] = df_sm.apply(lambda row : extract_emojis(row['tweet_text']), axis = 1)

df['extracted_emojis'].head(5)

In [ ]:
df.head(5)

In [ ]:
extracted_list = df_sm['extracted_emojis'].values.tolist()
print(extracted_list)
print(len(extracted_list))

*classification*:
if all emojis in `extracted_emojis` is positive - positive
if all emojis in `extracted_emojis` is negative - negative 
rest - randomly assign label
def score(extracted_list):


https://www.geeksforgeeks.org/python-check-if-one-list-is-subset-of-other/


In [ ]:
# pseudo Code
for lst in extracted_list:
  if lst is a subset of pos collection:
    position_to_cursor
    elif list is a subset of neg collection:
      neg
      else:
        random
#def score(extracted_list):